In [1]:
import os
import pandas as pd
from utils import partial_match_scores


root = "../datasets/myriadlama/llama3.2_1b_it"
# root = "../datasets/myriadlama/llama3.2_3b_it"
# root = "../datasets/myriadlama/qwen2.5_3b_it"
# root = "../datasets/myriadlama/qwen2.5_7b_it"
root = "../datasets/myriadlama/llama3.1_8b_it"


In [2]:
from utils import partial_match


confi_df = pd.read_feather(os.path.join(root, "confidence.feather"))
confi_df["sample_lemmas"] = confi_df["sample_lemmas"].apply(lambda xs: [list(x) for x in xs])
confi_df["answer_lemmas"] = confi_df["answer_lemmas"].apply(lambda xs: [list(x) for x in xs])


all_matches = []
all_predictions = []
all_answers = []
all_uuids = []
for uuid, sconfi_df in confi_df.groupby("uuid"):
    all_predictions.append(sconfi_df['greedy_lemma'].tolist())
    all_answers.append(sconfi_df['answer_lemmas'].tolist()[0])
    all_uuids.append(uuid)
    # matches, acc = partial_match_score(sdf['prediction'].tolist(), sdf["answers"].tolist())

for idx, predictions in enumerate(zip(*all_predictions)):
    matches = []
    for prediction, _gold_answers in zip(predictions, all_answers):
        score = partial_match(prediction, _gold_answers, False)
        matches.append(int(score))
    
    all_matches.append(matches)
    print(f"Paraphrase {idx}: {sum(matches)/len(matches):.4f}")


Paraphrase 0: 0.5520
Paraphrase 1: 0.4590
Paraphrase 2: 0.5780
Paraphrase 3: 0.5850
Paraphrase 4: 0.5210


In [3]:
for i in range(2, 6):
    or_match = sum(any(ms) for ms in zip(*all_matches[:i]))
    print(f"Ratio of or_match with {i} paraphrases: {or_match / len(all_matches[0]):.4f}")    
    

Ratio of or_match with 2 paraphrases: 0.6400
Ratio of or_match with 3 paraphrases: 0.7120
Ratio of or_match with 4 paraphrases: 0.7450
Ratio of or_match with 5 paraphrases: 0.7550


In [5]:
# for i in range(2, 6):
for i in [2]:
    weighted_avg_fn = os.path.join(root, f"ensemble_weighted_avg-{idx}.feather")
    df = pd.read_feather(weighted_avg_fn)
    df["answer_lemmas"] = df["answer_lemmas"].apply(lambda xs: [list(x) for x in xs])

    uuids = df["uuid"].tolist()
    answers = [ans for ans in df["answer_lemmas"].tolist()]
    predictions = [pred for pred in df["predict_lemma"].tolist()]
    ensemble_matches = []

    interesting_uuids = []
    for uuid, prediction, gold_answers in zip(uuids, predictions, answers):
        matched = partial_match(prediction, gold_answers, False)
        ensemble_matches.append(int(matched))

        sconfi_df = confi_df[confi_df["uuid"] == uuid]
        answers = sconfi_df["answer_lemmas"].tolist()[0]
        or_matched = False
        for _prediction in sconfi_df["greedy_lemma"].tolist():
            or_matched |= partial_match(_prediction, answers, False)

        if matched and not or_matched:
            print(uuid, prediction, gold_answers)
            interesting_uuids.append(uuid)

5d09424876ba7f4-5dc40def3637be0 ['nbc'] [['nbc'], ['national', 'broadcasting', 'company']]


In [6]:
confi_df[confi_df["uuid"] == interesting_uuids[0]]

,uuid,paraphrase,prompt,answers,greedy_predict,sample_predicts,confidence,greedy_lemma,sample_lemmas,answer_lemmas
2321,5d09424876ba7f4-5dc40def3637be0,"Hey, Landlord premiered on the network [MASK].",Predict the [MASK] in the sentence in one word...,"[NBC, National Broadcasting Company]",HBO,"[HBO, UGC, MTV, tvn, USA., Cooking Channel, MT...",0.15,[hbo],"[[hbo], [ugc], [mtv], [tvn], [usa, .], [cookin...","[[nbc], [national, broadcasting, company]]"
2329,5d09424876ba7f4-5dc40def3637be0,"[MASK] is the first air channel of Hey, Landlord.",Predict the [MASK] in the sentence in one word...,"[NBC, National Broadcasting Company]",1,"[Zhenhao, U1, War , Cathode Ray, Econ, Cooking...",0.00,[1],"[[zhenhao], [u1], [war], [cathode, ray], [econ...","[[nbc], [national, broadcasting, company]]"
2337,5d09424876ba7f4-5dc40def3637be0,"Which service originally aired Hey, Landlord? ...",Predict the [MASK] in the sentence in one word...,"[NBC, National Broadcasting Company]",CBS,"[NBC, CBS, Warner Bros, ================= NBC,...",0.29,[cbs],"[[nbc], [cbs], [warner, bros], [=, =, =, =, =,...","[[nbc], [national, broadcasting, company]]"
2345,5d09424876ba7f4-5dc40def3637be0,"Hey, Landlord was originally aired on [MASK].",Predict the [MASK] in the sentence in one word...,"[NBC, National Broadcasting Company]",BBC,"[NBC, UGC, MTV JAPAN show Hai Rei, tv, USA, CB...",0.11,[bbc],"[[nbc], [ugc], [mtv, japan, show, hai, rei], [...","[[nbc], [national, broadcasting, company]]"
2353,5d09424876ba7f4-5dc40def3637be0,"Hey, Landlord was initially launched on [MASK].",Predict the [MASK] in the sentence in one word...,"[NBC, National Broadcasting Company]",2013,"[YouTube, iPhone, Warner Bros Pictures., Faceb...",0.00,[2013],"[[youtube], [iphone], [warner, bros, pictures,...","[[nbc], [national, broadcasting, company]]"
